In [16]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

# temporary solution for relative imports in case sadl is not installed
# if sadl is installed, no need to use the line
import sys
import os
import inspect
from inspect import signature
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from SADL.static_data.algorithms import pyod

kwargs = {"algorithm_": "cblof","contamination":2, "label_parser": True}
model = pyod.PyodAnomalyDetection(**kwargs)

#print(model.algorithm_)
print(model.label_parser)
print(model.get_params())
#print(signature(model.__class__).parameters.values())

kwargs = {"algorithm_": "abod","contamination":2, "label_parser": False}
model.set_params(**kwargs)
print(model.label_parser)
print(model.get_params())


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': True, 'algorithm_': 'CBLOF', 'alpha': 0.9, 'beta': 5, 'check_estimator': False, 'clustering_estimator': None, 'contamination': 0.1, 'n_clusters': 8, 'n_jobs': None, 'random_state': None, 'use_weights': False}
True
{'label_parser': True, 'algorithm_': 'CBLOF', 'alpha': 0.9, 'beta': 5, 'check_estimator': False, 'clustering_estimator': None, 'contamination': 2, 'n_clusters': 8, 'n_jobs': None, 'random_state': None, 'use_weights': False}
{'label_parser': False, 'algorithm_': 'ABOD', 'contamination': 2, 'method': 'fast', 'n_neighbors': 5}
False
{'label_parser': False, 'algorithm_': 'ABOD', 'contamination': 2, 'method': 'fast', 'n_neighbors': 5}


Data generation

In [17]:
# Define the number of inliers and outliers
n_samples = 200
outliers_fraction = 0.25

# Compare given detectors under given settings
# Initialize the data
xx, yy = np.meshgrid(np.linspace(-7, 7, 100), np.linspace(-7, 7, 100))
n_inliers = int((1. - outliers_fraction) * n_samples)
n_outliers = int(outliers_fraction * n_samples)
ground_truth = np.zeros(n_samples, dtype=int)
ground_truth[-n_outliers:] = 1

# Show the statics of the data
print('Number of inliers: %i' % n_inliers)
print('Number of outliers: %i' % n_outliers)
print('Ground truth shape is {shape}. Outlier are 1 and inliers are 0.\n'.format(shape=ground_truth.shape))
print(ground_truth)

X1 = 0.3 * np.random.randn(n_inliers // 2, 2)
X2 = 0.3 * np.random.randn(n_inliers // 2, 2)
X = np.r_[X1, X2]
# Add outliers
X = np.r_[X, np.random.uniform(low=-6, high=6, size=(n_outliers, 2))]


Number of inliers: 150
Number of outliers: 50
Ground truth shape is (200,). Outlier are 1 and inliers are 0.

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Model Fitting - errors


In [18]:
%load_ext autoreload
%autoreload 2
kwargs = {"algorithm_": "abod","contamination":2, "label_parser": True, "n_neighbors": 4, "method": 'method'}
model = pyod.PyodAnomalyDetection(**kwargs)

# Fit the model using SADL Library - error en fit
model.fit(X)
kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": True, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)
#print(model.model)
# Fit the model using SADL Library - sin error
#model.fit(X)
scores_pred = model.decision_function(X) * -1
print(scores_pred)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': True, 'algorithm_': 'ABOD', 'contamination': 0.1, 'method': 'fast', 'n_neighbors': 5}
PYODerror fit(): ('method', 'is not a valid method')
For further reference please see: https://pyod.readthedocs.io/en/latest/
{'label_parser': True, 'algorithm_': 'ABOD', 'contamination': 2, 'method': 'method', 'n_neighbors': 4}
PYODerror decision_function(): This ABOD instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
For further reference please see: https://pyod.readthedocs.io/en/latest/


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

Model Fitting - OK


In [19]:
%load_ext autoreload
%autoreload 2
kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": True, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.model)
# Fit the model using SADL Library - sin error
model.fit(X)
scores_pred = model.decision_function(X) * -1
print(scores_pred)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': True, 'algorithm_': 'ABOD', 'contamination': 0.1, 'method': 'fast', 'n_neighbors': 4}
ABOD(contamination=0.1, method='fast', n_neighbors=4)
[7.42220263e+03 5.85865641e+03 2.21238516e+03 1.43488610e+04
 1.77982978e+03 1.81407952e+04 1.19262818e+03 3.75411710e+05
 1.27137359e+04 2.28985155e+02 7.23920849e+02 8.84861268e+03
 2.64749550e+01 4.92409478e+04 1.49234376e+02 1.89131682e+05
 5.56964572e+04 3.97319955e+01 3.14535264e+03 5.62936824e+02
 2.24495004e+05 6.47240520e+04 5.38467778e+04 6.25351400e+04
 1.65832960e+04 3.86651765e+05 1.69872408e+01 1.65024890e+03
 1.20762066e+04 1.97802832e+05 3.64770027e+04 6.33087663e+05
 2.46434619e+05 7.78997628e+04 2.83294775e+03 1.38148838e+04
 2.24618230e+04 3.24417711e+03 3.20838303e+04 6.18094500e+04
 4.15467911e+04 2.66570097e+01 1.19420338e+04 7.87742000e+04
 8.96468503e+04 5.99054280e+06 1.94765706e+02 1.06514495e+03
 9.18984632e+04 3.66387

Model Predict w/ label_parser function

In [21]:
%load_ext autoreload
%autoreload 2

def function(X):
    print("Estoy en funcion")
    return X < 0.5

kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": function, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.get_params())
model.predict(X)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': <function function at 0x7fcf4f92fd80>, 'algorithm_': 'ABOD', 'contamination': 0.1, 'method': 'fast', 'n_neighbors': 4}
Estoy en funcion


array([[ True,  True],
       [ True,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True,  True],
       [ Tr

Model Predict wo/ label_parser

In [22]:
%load_ext autoreload
%autoreload 2


def function(X):
    print("Estoy en funcion")
    return X < 0.5

kwargs = {"algorithm_": "abod","contamination":0.1, "label_parser": None, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.get_params().keys())
model.fit(X)
model.predict(X)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_keys(['label_parser', 'algorithm_', 'contamination', 'method', 'n_neighbors'])


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1])

In [52]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pyod.models.lscp import LSCP
from pyod.models.lof import LOF
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from SADL.static_data.algorithms.pyod import PyodAnomalyDetection

kwargs_lof = {"algorithm_": "lof"}

detector_list = [LOF(), LOF()]

kwargs = {"algorithm_": "lscp","detector_list":detector_list}
model = pyod.PyodAnomalyDetection(**kwargs)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': None, 'algorithm_': 'LSCP', 'contamination': 0.1, 'detector_list': [LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2)], 'local_max_features': 1.0, 'local_region_size': 30, 'n_bins': 10, 'random_state': None}
{'label_parser': None, 'algorithm_': 'LSCP', 'contamination': 0.1, 'detector_list': [LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2)], 'local_max_features': 1.0, 'local_region_size': 30, 'n_bins': 10, 'random_state'